In [45]:
import pandas as pd
from joblib import load
from sklearn.preprocessing import OneHotEncoder

In [125]:
base_path = '/Users/Neezy/Desktop/ds_projects/m5_forecasting/data/'
sales = pd.read_pickle(base_path + 'sales_init_pp.pkl').set_index('id')
sales.head()

,item_id,dept_id,cat_id,store_id,state_id,day,sales
id,,,,,,,
HOBBIES_1_001_CA_1_validation,1,1,HOBBIES,1,CA,1,0
HOBBIES_1_002_CA_1_validation,2,1,HOBBIES,1,CA,1,0
HOBBIES_1_003_CA_1_validation,3,1,HOBBIES,1,CA,1,0
HOBBIES_1_004_CA_1_validation,4,1,HOBBIES,1,CA,1,0
HOBBIES_1_005_CA_1_validation,5,1,HOBBIES,1,CA,1,0


In [126]:
submission_grid = pd.read_csv(base_path + 'sample_submission.csv')['id']
submission_grid.head()

0    HOBBIES_1_001_CA_1_validation
1    HOBBIES_1_002_CA_1_validation
2    HOBBIES_1_003_CA_1_validation
3    HOBBIES_1_004_CA_1_validation
4    HOBBIES_1_005_CA_1_validation
Name: id, dtype: object

In [127]:
sales_grid = sales[sales['day'] == 1].drop(['day', 'sales'], axis=1)
sales_grid.head()

,item_id,dept_id,cat_id,store_id,state_id
id,,,,,
HOBBIES_1_001_CA_1_validation,1,1,HOBBIES,1,CA
HOBBIES_1_002_CA_1_validation,2,1,HOBBIES,1,CA
HOBBIES_1_003_CA_1_validation,3,1,HOBBIES,1,CA
HOBBIES_1_004_CA_1_validation,4,1,HOBBIES,1,CA
HOBBIES_1_005_CA_1_validation,5,1,HOBBIES,1,CA


In [128]:
pred_grid = sales_grid[sales_grid.index.isin(list(submission_grid))]
pred_grid = pred_grid.reset_index()
pred_grid.head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,1,1,HOBBIES,1,CA
1,HOBBIES_1_002_CA_1_validation,2,1,HOBBIES,1,CA
2,HOBBIES_1_003_CA_1_validation,3,1,HOBBIES,1,CA
3,HOBBIES_1_004_CA_1_validation,4,1,HOBBIES,1,CA
4,HOBBIES_1_005_CA_1_validation,5,1,HOBBIES,1,CA


In [129]:
days = []
for day in range(1914, 1942):
    days.append(day)

In [130]:
for col in days:
    pred_grid[col] = 0

In [131]:
id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
pred_grid = pred_grid.melt(id_vars=id_vars, var_name='day', value_name='sales')

In [133]:
one_hot = load('../models/one_hot_encoder.joblib')

In [134]:
# Split categorical variables
cat_vars = pred_grid[['dept_id', 'cat_id', 'store_id', 'state_id']]
other_vars = pred_grid[['day']]
id_var = pred_grid['id']
# One hot encoding
cat_vars_oh = one_hot.transform(cat_vars).toarray()
cat_vars_oh = pd.DataFrame(
    cat_vars_oh, columns=one_hot.get_feature_names(cat_vars.columns))
cat_vars_oh.index = cat_vars.index

# Append Y again
pred_x_oh = pd.concat([cat_vars_oh, other_vars], axis=1)
pred_x_oh.head()

,dept_id_2,dept_id_3,cat_id_HOBBIES,cat_id_HOUSEHOLD,store_id_2,store_id_3,store_id_4,state_id_TX,state_id_WI,day
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1914
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1914
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1914
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1914
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1914


In [135]:
rf = load('../models/rf_v_1_0.joblib')

In [136]:
pred_grid['sales'] = rf.predict(pred_x_oh)

In [137]:
pred_grid['sales'] = pred_grid['sales'].round().astype(int)

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
658277,FOODS_3_518_TX_2_validation,518,3,FOODS,2,TX,1935,2
385785,HOUSEHOLD_2_516_TX_3_validation,516,2,HOUSEHOLD,3,TX,1926,1
372885,HOUSEHOLD_1_349_CA_3_validation,349,1,HOUSEHOLD,3,CA,1926,6
83042,HOUSEHOLD_1_159_WI_1_validation,159,1,HOUSEHOLD,1,WI,1916,2
372701,HOUSEHOLD_1_163_CA_3_validation,163,1,HOUSEHOLD,3,CA,1926,6
588593,HOBBIES_1_141_CA_4_validation,141,1,HOBBIES,4,CA,1933,1
89354,HOUSEHOLD_1_376_WI_3_validation,376,1,HOUSEHOLD,3,WI,1916,1
376936,FOODS_2_082_CA_4_validation,82,2,FOODS,4,CA,1926,1
831002,FOODS_1_064_CA_3_validation,64,1,FOODS,3,CA,1941,1
251923,FOODS_2_078_CA_3_validation,78,2,FOODS,3,CA,1922,2


In [145]:
final_preds = pred_grid.pivot(index='id', columns='day', values='sales').reset_index()
final_preds.rename_axis(None, axis=1, inplace=True)

In [163]:
col_names = ['id']
for col_name in range(1, 29):
    col_names.append('F' + str(col_name))

In [164]:
final_preds.columns = col_names

In [185]:
submission_grid = pd.read_csv(base_path + 'sample_submission.csv')

In [177]:
submission = final_preds.append(submission_grid).reset_index(drop=True)

In [188]:
submission.to_csv('../submissions/rf_v_1_0.csv', header=True, index=False)